In [2]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd 
import os
import re 
import numpy as np


from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='/Users/alexaj/anaconda3/bin/chromedriver')

In [2]:
url = "https://degreeworks.banner.marist.edu/dashboard/dashboard"
driver.get(url)

In [3]:
#driver.implicitly_wait(5)
contFrame = driver.find_element_by_css_selector('html > frameset > frame:nth-child(4)')
driver.switch_to.frame(contFrame)
#frBC = driver.find_element_by_xpath('/html/frameset')
bodyFrame = driver.find_element_by_name('frBody')
driver.switch_to.frame(bodyFrame)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

In [5]:
#taking majors titles, minor titles, pathway
headTitles = soup.find_all("td", attrs={"class": "BlockHeadTitle"})
headTitle = []
for head in headTitles:
    headTitle.append(head.text)

majorArray = [x for x in headTitle if 'Major' in x]
concentrations = [x for x in headTitle if 'Concentration' in x]
minorArray = [x for x in headTitle if 'Minor' in x]
degreeName = [x for x in headTitle if 'Degree' in x]
pwayName = [x for x in headTitle if 'Pathway' in x][0]

In [6]:
#check the above variables
creditTitles = []
creditTitles = degreeName + majorArray + concentrations + minorArray 
creditTitles

['Degree in Bachelor of Science',
 'Major in Computer Science',
 'Concentration in Software Development',
 'Minor in Math']

In [8]:
#code for required courses 
jsonObjects = []

def req(soup):
    reqStrings = soup.find_all("td", attrs={"class": "RuleLabelTitleNeeded"})

    reqCourses = []
    for reqString in reqStrings:
        reqCourses.append(reqString.text)

    coreReqs = [x for x in reqCourses if '3 credits' in x]
    #coreReqs
    coreReqDf = pd.DataFrame(coreReqs)
    coreReqDf.columns = ['Core Requirement']

    # coreReqFileJSON = 'SerenityCoreReqs.json'
    coreJSON = coreReqDf.to_json(orient='records')
    return coreJSON

# jsonObjects.append(req(soup))
# jsonObjects
req(soup)

'[{"Core Requirement":"3 credits Ethics\\/Applied Ethics\\/Religious Studies"}]'

In [182]:
#HONORS! Special case 
# css selector for honors frame : frmAudit > table:nth-child(38) > tbody > tr > td > table

In [ ]:
pathwayClassesDri = driver.find_element_by_css_selector('#frmAudit > table:nth-child(28) > tbody > tr > td > table > tbody > tr:nth-child(5) > td.RuleLabelData > table')
pathwaySoup = BeautifulSoup(pathwayClassesDri.get_attribute('innerHTML'), "html5lib")

pathwayTable = pathwaySoup.find_all('table')[0]
pathwayDf = pd.read_html(str(pathwayTable))[0]
pathwayDf.columns = ['Course No', 'Course Title', 'Grade', 'Credits', 'Semester']
pathwayDf

# pathwayFileJSON = 'SerenityPathway.json'
# pathwayDf.to_json(pathwayFileJSON, orient="records")
# pathwayFileCSV = 'SerenityPathway.csv'
# pathwayDf.to_json(pathwayFileCSV, index=False)

In [49]:
#Code for scraped data from Student View information
studentTable = soup.find("table", attrs={"class": "Inner"})

studentInfoDf = pd.read_html(str(studentTable))[0]

primValue = list(studentInfoDf[0])
primInfo = list(studentInfoDf[1])
secondColumn = list(studentInfoDf[2])
thirdColumn = list(studentInfoDf[3])

finalValue = primValue + secondColumn
finalInfo = primInfo + thirdColumn

studentDict = dict(zip(finalValue, finalInfo))

# studentInfoFinalDf = pd.DataFrame.from_dict(studentDict, orient='columns')

# studentInfoFileJSON = 'SerenityStudentInfo.json'
# studentInfoFinalDf.to_json(studentInfoFileJSON, orient='records')
import json
studentJSONStr = json.dumps(studentDict)
studentJSONStr

'{"Student": "Javellana, Alexa", "ID": "20068940", "Classification": "Junior", "Advisor": "Cannistra, Robert M", "Overall GPA": "3.476", "Student Type": "Traditional Continuing", "Term Location": "Poughkeepsie", "Level": "Undergraduate", "Degree": "B.S.", "College": "Computer Science & Mathematics", "Major": "Computer Science", "Minor": "Math", "Concentration": "CompSci: Software Development", "NaN": NaN}'

In [32]:
semYear = datetime.date.today().year
semMonth = datetime.date.today().month
semSsn = ""

if (semMonth >= 1 and semMonth <= 5):
  semSsn = "Spring"
elif (semMonth >= 6 and semMonth <= 8):
  semSsn = "Summer"
elif (semMonth >= 9 and semMonth <= 12):
  semSsn = "Fall"
elif (semMonth == 1):
  semSsn = "Winter"

semSsn + " " + str(semYear)

'Fall 2018'

In [87]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})

if(len(currClassHTML) > 2):
    currClassHTML = currClassHTML[2]
else:
    currClassHTML = currClassHTML[1]

# currClassHTML
#classNames

className = currClassHTML.find_all("td", attrs={"class": "SectionCourseTitle"})
classNames = []

for classes in className:
    classNames.append(classes.text)
    
#classNumbers

classNo = currClassHTML.find_all("td", attrs={"class": "ClassesAppliedClasses"})
classNos = []

for no in classNo:
    no = no.text.replace(u'\xa0', u' ')
    classNos.append(no)

#creditsEach

noCredits = currClassHTML.find_all("td", attrs={"class": "SectionCourseCredits"})
curCredits = []

for noCreds in noCredits:
    curCredits.append(noCreds.text)
    
# curCredits
currClassDataStack = np.vstack((classNos, classNames, curCredits)).T
currClassDataStack = currClassDataStack.tolist()

currClassInfoDf = pd.DataFrame(currClassDataStack,columns=['Course No', 'Course Title', 'Credit Value'])
# currClassFileCSV = 'SerenityCurrClass.csv'
# currClassInfoDf.to_csv(currClassFileCSV, index=False)

# studentInfoFileJSON = 'SerenityStudentInfo.json'
# studentView.to_json(studentInfoFileJSON)
currClassInfoDf

,Course No,Course Title,Credit Value
0,CMPT 105N,MS EXCEL,1
1,CMPT 410N,SYSTEMS ADMINST-MGTMNT,4
2,CMPT 430L,TECHNOLOGY+ENTREPRENEURSHIP,3
3,CMPT 475N,CS PROJECT,3
4,CMPT 476N,CS PROJECT II,1
5,MATH 130L,INTRO-STATIST I,3
6,PHED 132N,BOXING,1


In [20]:
# currClassesDri = driver.find_element_by_css_selector('#frmAudit > table:nth-child(67)')
# currClassSoup = BeautifulSoup(currClassesDri.get_attribute('innerHTML'), "html5lib")

currClassDataStack = np.vstack((classNos, classNames, curCredits)).T
currClassDataStack = currClassDataStack.tolist()

currClassInfoDf = pd.DataFrame(currClassDataStack,columns=['Course No', 'Course Title', 'Credit Value'])
# currClassFileCSV = 'SerenityCurrClass.csv'
# currClassInfoDf.to_csv(currClassFileCSV, index=False)

# studentInfoFileJSON = 'SerenityStudentInfo.json'
# studentView.to_json(studentInfoFileJSON)

In [88]:
#This code parses the data on DegreeWorks by td class, BlockHeadSubd
#Code parses credits into total and completed arrays
subData = soup.find_all("td", attrs={"class": "BlockHeadSubData"})
#dataNP = array of subData that is not parsed, but converted from obj to str
subDataNP = []
for subD in subData:
    subDataNP.append(subD.text)
    #objToText

#dataP = array subData that is/will be parsed
subDataP = []
for subDataN in subDataNP:
    string = subDataN.split("\xa0 ",1)[1]
    if string:
        subDataP.append(string)

    
credits = []

for subData in subDataP:
    if not any(s in subData for s in ('-', '.')):
        credits.append(subData)

    
creditTitles = []
creditTitles = degreeName + majorArray + concentrations + minorArray 
if (len(creditTitles) * 2 != len(credits)):
    creditTitles = degreeName + concentrations + minorArray 
        
        
totalCredits = []
completedCredits = []
i = 0
while i < len(credits):
    if i % 2 == 0:
        totalCredits.append(credits[i])
        i += 1
    else:
        completedCredits.append(credits[i])
        i += 1
        
creditsProgress = np.vstack((creditTitles, completedCredits, totalCredits)).T
creditsProgress = creditsProgress.tolist()
progressDf = pd.DataFrame(creditsProgress,columns=['Title', 'Credits Completed', 'Total Needed'])
progressDf

,Title,Credits Completed,Total Needed
0,Degree in Bachelor of Science,88,120
1,Major in Information Technology and Systems,67,73
2,Concentration: Information Technology,18,20
3,Minor in Computer Science,24,22
4,Minor in Cybersecurity,15,23
5,Minor in Information Systems,23,23


In [282]:
credits

['120', '88', '73', '67', '20', '18', '22', '24', '23', '15', '23', '23']

In [170]:
#credit progress data dictionary JSON
majMinFileJSON = 'SerenityMajMin.json'
progressDf.to_json(majMinFileJSON, orient='records')

In [201]:
ppmInfo = np.vstack((ppmCountry, ppm)).T
ppmInfo = ppmInfo.tolist()

In [202]:
df = pd.DataFrame(ppmInfo,columns=['By country', 'Page views per minute'])

In [204]:
df

,By country,Page views per minute
0,GB,0.0176
1,US,0.011
2,IE,0.00347
3,DE,0.00288
4,AT,0.00278
5,BR,0.00198
6,SK,0.00169
7,HU,0.00139
8,RU,0.00129
9,VE,0.000794
